## Directory

In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\Marina\\Desktop\\ML Operations\\0 - KrishNaik Course\\21_end_to_end_nlp_project_with_huggingface_and_transformers\\my_project'

## 1. Config.yaml

Vamos fazer isso, criando o artifact folder e a classe data_ingestion

## 2. Params.yaml

## 3. Config entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig():
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

## 4. Configuration Manager

Vamos criar umas constanstes 

In [ ]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                config_path= CONFIG_FILE_PATH,
                params_path= PARAMS_FILE_PATH ):
        
        self.configurations = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.configurations.artifacts_root]) # cria o /artifacts

    # def get_somestep(self)->SomeStepConfig:
        # config = self.configurations.somestep

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        data_ingestion_config = self.configurations.data_ingestion
        
        create_directories([data_ingestion_config.root_dir]) # cria o /artifacts

        return data_ingestion_config


In [ ]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()

[ 2024-11-12 10:47:54,163 ] - 28 summarizerlogger - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-11-12 10:47:54,169 ] - 28 summarizerlogger - INFO - yaml file: params.yaml loaded successfully
[ 2024-11-12 10:47:54,173 ] - 46 summarizerlogger - INFO - created directory at: artifacts
[ 2024-11-12 10:47:54,176 ] - 46 summarizerlogger - INFO - created directory at: artifacts/data_ingestion


In [ ]:
print(data_ingestion_config)

{'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}


In [ ]:
print(data_ingestion_config.local_data_file)

artifacts/data_ingestion/data.zip


## 5. Update the components- Data Ingestion,Data Transformation, Model Trainer

In [ ]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger

class DataIngestion():
    def __init__(self, 
                data_ingestion_config: DataIngestionConfig):

        self.data_ingestion_config = data_ingestion_config

    def download_data(self):
        """Downloads data from the source URL to the specified local path."""
        if not os.path.exists(self.data_ingestion_config.local_data_file):
            try:
                logger.info(f"Downloading data from {self.data_ingestion_config.source_URL}")
                request.urlretrieve(self.data_ingestion_config.source_URL, self.data_ingestion_config.local_data_file)
                logger.info(f"Data downloaded successfully to {self.data_ingestion_config.local_data_file}")
            except Exception as e:
                logger.error(f"Error downloading data: {e}")
                raise e
        else:
            logger.info(f"Data file already exists at {self.data_ingestion_config.local_data_file}")

    def extract_data(self):
        """Extracts the zip file to the specified directory."""
        if not os.path.exists(self.data_ingestion_config.unzip_dir):
            os.makedirs(self.data_ingestion_config.unzip_dir, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.data_ingestion_config.local_data_file, 'r') as zip_ref:
                logger.info(f"Extracting data to {self.data_ingestion_config.unzip_dir}")
                zip_ref.extractall(self.data_ingestion_config.unzip_dir)
                logger.info("Data extracted successfully.")
        except Exception as e:
            logger.error(f"Error extracting data: {e}")
            raise e

    def initiate_data_ingestion(self):
        """Orchestrates the download and extraction of data."""
        self.download_data()
        self.extract_data()

In [10]:
data_ingestion_object = DataIngestion(data_ingestion_config)
data_ingestion_object.initiate_data_ingestion()

[ 2024-11-12 11:00:28,927 ] - 17 summarizerlogger - INFO - Downloading data from https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
[ 2024-11-12 11:00:31,727 ] - 19 summarizerlogger - INFO - Data downloaded successfully to artifacts/data_ingestion/data.zip
[ 2024-11-12 11:00:31,759 ] - 33 summarizerlogger - INFO - Extracting data to artifacts/data_ingestion
[ 2024-11-12 11:00:31,946 ] - 35 summarizerlogger - INFO - Data extracted successfully.


## 6. Modularizar o Código

O `3.` vai para `src\textSummarizer\entity\__init__.py`

O `4.` vai para `src\textSummarizer\config\configuration.py`

O `5.` vai para `src\textSummarizer\components\data_ingestion.py`

Modularizamos criando uma pipeline (classe) em `stage_1_data_ingestion_pipeline.py`, com o que usamos para rodar o código

Jogar a Pipeline para `main.py`